In [1]:
import win32com.client
import pandas as pd
from datetime import datetime
from datetime import timedelta
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
instCpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
instStockChart = win32com.client.Dispatch('CpSysDib.StockChart')


In [2]:
# 크레온 API 연결 확인
# <리턴값 종류 확인 필요>
def check_connection():
    if instCpCybos.IsConnect == 1:
        print('정상적으로 연결')
    else:
        print('연결 실패')
        
check_connection()

정상적으로 연결


In [3]:
# 종목 이름으로 찾기
def search_by_name(stock_name):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(1,i) == stock_name:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_name('삼성전자')

['A005930', '삼성전자']

In [4]:
# 특정 코드의 종목 찾기
def search_by_code(stock_code):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(0,i) == stock_code:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_code('A005930')

['A005930', '삼성전자']

In [5]:
# 종목명 전체 리스트 가져오기
# <선물이랑 ETF 쪽은 구분 필요>
def get_stock_list():
    count = instCpStockCode.GetCount()
    stock_list = []
    
    for i in range(count):
        stock_list.append(instCpStockCode.GetData(1,i))
    
    return stock_list

# get_stock_list()

In [6]:
# print(instCpCodeMgr.GetStockListByMarket(1))

In [7]:
# 전체 종목 가져오기
codelist = instCpCodeMgr.GetStockListByMarket(1)
kospi = {}
for code in codelist:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
# kospi

In [8]:
# 
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    # print(i, code, secondCode, name)

In [9]:
# 
code_list = []
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    code_list.append(code)
# print(code_list)


In [10]:
import time
# 특정 범위 일자 종목 데이터 가져오기
def get_stock_info(stock_code, start_day, end_day, type):
    
    instStockChart.SetInputValue(0, stock_code) # 종목명
    instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, start_day) # 요청 시작일
    instStockChart.SetInputValue(2, end_day) # 요청 종료일
    # instStockChart.SetInputValue(4, request_num) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 8 : 거래량, 9 : 거래대금, 13 : 시가총액
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
    instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info

donghwa = get_stock_info('A005930', '20221029', '20221031', 'm')

In [12]:
import time
# 전체 일자 분봉 종목 데이터 가져오기
def get_stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=1)
    
    stock_df = get_stock_info(stock_code, end_day, end_day, type) # 첫번째 DF 생성 => pre_day '20221028' end_day '20221103'
    
    stock_name = search_by_code(stock_code)

    for day in range(740): # range 범위 수정하지 말 것
        transfer_pre_day = datetime.strptime(end_day, day_format) # '20221028' '20221023'
        pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format) # '20221023' '20221018'

        info_second = get_stock_info(stock_code, pre_day, pre_day, type) # '20221023' '20221027'

        time.sleep(0.15)
        print(pre_day)

        stock_df = pd.concat([stock_df, info_second], axis=0)
        info_second = pd.DataFrame()
        print(stock_df.shape)
        print(stock_name, 'Count : {0}, 남은 요청 횟수 :{1}'.format(day, instCpCybos.GetLimitRemainCount(1))) 
    
    stock_df.to_csv('../data/{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

# samsung = get_stock_info_auto('A005930', '20221101', 'm')
# samsung

In [ ]:
import time
for code in code_list[:]:
    get_stock_info_auto(code, '20221103', 'm')


In [95]:
get_stock_info_auto('A005930', '20221103', 'm')

20221030 20221031
(1180, 24)
['A005930', '삼성전자'] Count : 0, 남은 요청 횟수 :58
20221028 20221029
(1561, 24)
['A005930', '삼성전자'] Count : 1, 남은 요청 횟수 :57
20221026 20221027
(2323, 24)
['A005930', '삼성전자'] Count : 2, 남은 요청 횟수 :56
20221024 20221025
(3085, 24)
['A005930', '삼성전자'] Count : 3, 남은 요청 횟수 :55
20221022 20221023
(3085, 24)
['A005930', '삼성전자'] Count : 4, 남은 요청 횟수 :54
20221020 20221021
(3847, 24)
['A005930', '삼성전자'] Count : 5, 남은 요청 횟수 :53
20221018 20221019
(4609, 24)
['A005930', '삼성전자'] Count : 6, 남은 요청 횟수 :52
20221016 20221017
(4990, 24)
['A005930', '삼성전자'] Count : 7, 남은 요청 횟수 :51
20221014 20221015
(5371, 24)
['A005930', '삼성전자'] Count : 8, 남은 요청 횟수 :60
20221012 20221013
(6133, 24)
['A005930', '삼성전자'] Count : 9, 남은 요청 횟수 :59
20221010 20221011
(6514, 24)
['A005930', '삼성전자'] Count : 10, 남은 요청 횟수 :58
20221008 20221009
(6514, 24)
['A005930', '삼성전자'] Count : 11, 남은 요청 횟수 :57
20221006 20221007
(7276, 24)
['A005930', '삼성전자'] Count : 12, 남은 요청 횟수 :56
20221004 20221005
(8038, 24)
['A005930', '삼성전자']

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221103,1530,59200,59200,59200,59200,0,1859760,110097790000,5944330,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1520,59500,59700,59500,59700,0,193177,11510350000,5944330,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1519,59500,59600,59500,59600,0,49046,2921250000,5893859,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1518,59600,59600,59500,59600,0,20663,1230500000,5874904,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1517,59500,59600,59500,59600,0,20021,1192690000,5864749,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,59900,60000,59800,59800,0,450531,26982330000,730219,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,904,59900,60000,59900,59900,0,159408,9552240000,494948,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,903,60000,60000,59900,59900,0,41872,2509660000,372455,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,902,59900,60100,59900,60000,0,172825,10367550000,345917,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0


In [36]:
test.index[-1]

'20221102'

In [96]:
test = pd.read_csv('../data/005930_삼성전자.csv', encoding='utf-8-sig')
test['Unnamed: 0'].value_counts()

20221103    381
20210705    381
20210616    381
20210617    381
20210618    381
           ... 
20220314    381
20201027    381
20210104    321
20220103    321
20221101     37
Name: Unnamed: 0, Length: 500, dtype: int64